In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


k_fold = 10
seeds = [192760, 1393659, 1269293, 138973, 931308, 1088652, 1361151, 1456105, 820535, 1240839]


In [3]:
data_train_path = "Dataset/data_train.csv"
data_test_path = "Dataset/data_target_users_test.csv"

In [4]:
df = pd.read_csv(data_train_path)
df_test = pd.read_csv(data_test_path)

df.columns = ["UserID", "ItemID", "Interaction"]
df_test.columns = ["UserID"]

In [5]:
df

,UserID,ItemID,Interaction
0,1,7,1.0
1,1,15,1.0
2,1,16,1.0
3,1,133,1.0
4,1,161,1.0
...,...,...,...
478725,13024,13605,1.0
478726,13024,13823,1.0
478727,13024,15122,1.0
478728,13024,18185,1.0


In [6]:
metric_to_optimize="MAP"
cutoff_to_optimize=10

In [7]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [8]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample


urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation = []
urm_test = []
urm_train = []
urm_validation = []

for i in range(len(seeds)):
    a, b = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80,seed=seeds[i])
    c, d = split_train_in_two_percentage_global_sample(a, train_percentage = 0.80,seed=seeds[i])
    urm_train_validation.append(a)
    urm_test.append(b)
    urm_train.append(c)
    urm_validation.append(d)



In [10]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = []
for u in urm_validation:
    evaluator_validation.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2581 (20.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2598 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2621 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2601 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2649 (21.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2614 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2608 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2586 (20.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2593 

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = []
for u in urm_test:
    evaluator_test.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2146 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2177 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2157 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2132 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2149 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2129 (16.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2228 (17.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2183 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2140 

In [12]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender

recommenderClass = MultiThreadSLIM_SLIMElasticNetRecommender



In [13]:
import optuna

# Replace the following with your MySQL database connection details
host = "db-mysql-nyc3-14396-do-user-15539286-0.c.db.ondigitalocean.com"
port = 25060
database_name = "recsys_optuna"
username = "doadmin"
password = "AVNS_EIQ1D-GbDGJWZPsN3Mt"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
storage = optuna.storages.RDBStorage(url=mysql_url)

In [14]:
study = optuna.create_study(study_name="SLIM_Hybrid_last", direction="maximize", storage=storage, load_if_exists=True, sampler=optuna.samplers.TPESampler())

[I 2024-01-09 22:05:11,183] A new study created in RDB with name: SLIM_Hybrid_last


In [15]:
def objective(trial):

    params = {
        'alpha': trial.suggest_float('alpha', 1e-5, 1e-2),
        'l1_ratio': trial.suggest_float('l1_ratio', 0.01, 0.5),
        'topK': trial.suggest_int('topK', 5, 10000),
    }

    MAP = 0
    for i in range(k_fold):
        recommender = recommenderClass(urm_train[i])
        recommender.fit(workers=8, **params)

        result_dict, _ = evaluator_validation[i].evaluateRecommender(recommender)

        MAP += result_dict["MAP"].item()
    
    MAP /= k_fold
    return MAP

In [16]:
study.optimize(objective, n_trials=200)

SLIMElasticNetRecommender: URM Detected 508 ( 4.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 332 ( 1.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:32<00:00, 676.26it/s]


EvaluatorHoldout: Processed 10057 (100.0%) in 4.13 sec. Users per second: 2432
SLIMElasticNetRecommender: URM Detected 440 ( 3.5%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 325 ( 1.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:32<00:00, 683.46it/s] 


EvaluatorHoldout: Processed 10040 (100.0%) in 3.70 sec. Users per second: 2711
SLIMElasticNetRecommender: URM Detected 479 ( 3.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 345 ( 1.6%) items with no interactions.


...

EvaluatorHoldout: Processed 10037 (100.0%) in 3.36 sec. Users per second: 2991
SLIMElasticNetRecommender: URM Detected 449 ( 3.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 320 ( 1.4%) items with no interactions.


100%|█████████▉| 22216/22222 [00:29<00:00, 764.10it/s] 


EvaluatorHoldout: Processed 9989 (100.0%) in 4.19 sec. Users per second: 2386
SLIMElasticNetRecommender: URM Detected 460 ( 3.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 328 ( 1.5%) items with no interactions.


 50%|████▉     | 11048/22222 [00:15<00:14, 753.95it/s][W 2024-01-10 09:25:44,887] Trial 189 failed with parameters: {'alpha': 0.0013916340441182929, 'l1_ratio': 0.14368546823180048, 'topK': 8177} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/multiprocessing/pool.py", line 851, in next
    item = self._items.popleft()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_255065/2072543649.py", line 12, in objective
    recommender.fit(workers=8, **params)
  File "/home/japo/RecSys-Challenge/Recommenders/SLIM/SLIMElasticNetRecommender.py", line 240, in fit
    for values_, rows_, cols_ in pool.imap_unordered(_pfi

KeyboardInterrupt: 

In [17]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = recommenderClass(urm_train[i])
    recommender.fit(workers=8, **study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict["MAP"].item()

best_MAP_test /= k_fold
best_MAP_test

SLIMElasticNetRecommender: URM Detected 508 ( 4.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 332 ( 1.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:39<00:00, 559.02it/s]


EvaluatorHoldout: Processed 10492 (100.0%) in 4.61 sec. Users per second: 2274
SLIMElasticNetRecommender: URM Detected 440 ( 3.5%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 325 ( 1.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:40<00:00, 549.45it/s]


EvaluatorHoldout: Processed 10461 (100.0%) in 4.96 sec. Users per second: 2109
SLIMElasticNetRecommender: URM Detected 479 ( 3.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 345 ( 1.6%) items with no interactions.


...

100%|█████████▉| 22216/22222 [00:37<00:00, 585.95it/s]


EvaluatorHoldout: Processed 10498 (100.0%) in 4.28 sec. Users per second: 2450
SLIMElasticNetRecommender: URM Detected 460 ( 3.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 348 ( 1.6%) items with no interactions.


100%|█████████▉| 22216/22222 [00:39<00:00, 560.76it/s]


EvaluatorHoldout: Processed 10451 (100.0%) in 4.21 sec. Users per second: 2482


0.0376160888619678

In [18]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = recommenderClass(urm_train_validation[i])
    recommender.fit(workers=8,**study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict["MAP"].item()

best_MAP_test /= k_fold
best_MAP_test

SLIMElasticNetRecommender: URM Detected 249 ( 2.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 113 ( 0.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:47<00:00, 472.17it/s]


EvaluatorHoldout: Processed 10492 (100.0%) in 5.18 sec. Users per second: 2027
SLIMElasticNetRecommender: URM Detected 191 ( 1.5%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 101 ( 0.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:43<00:00, 506.13it/s]


EvaluatorHoldout: Processed 10461 (100.0%) in 4.85 sec. Users per second: 2157
SLIMElasticNetRecommender: URM Detected 235 ( 1.9%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 105 ( 0.5%) items with no interactions.


...

100%|█████████▉| 22216/22222 [00:34<00:00, 648.77it/s]


EvaluatorHoldout: Processed 10498 (100.0%) in 3.95 sec. Users per second: 2658
SLIMElasticNetRecommender: URM Detected 217 ( 1.7%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 113 ( 0.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:34<00:00, 640.60it/s]


EvaluatorHoldout: Processed 10451 (100.0%) in 3.83 sec. Users per second: 2726


0.04939886597488928